# K-Means Clustering - Credit Card Dataset

Este notebook contiene el análisis completo del modelo K-Means para agrupar clientes de tarjetas de crédito en clusters.


## 1. Importación de Librerías


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import silhouette_score, davies_bouldin_score
from sklearn.decomposition import PCA
import pickle
import warnings
warnings.filterwarnings('ignore')

sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (12, 6)


## 2. Carga y Exploración de Datos


In [ ]:
# Cargar el dataset
df = pd.read_csv('CC GENERAL.csv')

# Información básica
print("Shape del dataset:", df.shape)
print("\nPrimeras filas:")
df.head()


## 3. Preprocesamiento de Datos


In [ ]:
# Eliminar columna CUST_ID si existe
if 'CUST_ID' in df.columns:
    df = df.drop('CUST_ID', axis=1)

# Manejar valores nulos
print("Valores nulos por columna:")
print(df.isnull().sum())

# Rellenar valores nulos con la mediana
df = df.fillna(df.median())

# Seleccionar solo features numéricas
numeric_cols = df.select_dtypes(include=[np.number]).columns.tolist()
X = df[numeric_cols]

print(f"\nFeatures numéricas seleccionadas: {len(numeric_cols)}")
print(f"Shape final: {X.shape}")


In [ ]:
# Estandarizar los datos
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

print("Datos estandarizados exitosamente")
print(f"Shape: {X_scaled.shape}")


## 4. Elección del Número Óptimo de Clusters

### 4.1 Método del Codo (Elbow Method)


In [ ]:
# Calcular inercia para diferentes valores de k
inertias = []
K_range = range(2, 11)

for k in K_range:
    kmeans = KMeans(n_clusters=k, random_state=42, n_init=10)
    kmeans.fit(X_scaled)
    inertias.append(kmeans.inertia_)

# Visualizar método del codo
plt.figure(figsize=(10, 6))
plt.plot(K_range, inertias, 'bo-', linewidth=2, markersize=8)
plt.xlabel('Número de Clusters (k)', fontsize=12)
plt.ylabel('Inercia', fontsize=12)
plt.title('Método del Codo - Elección de k óptimo', fontsize=14, fontweight='bold')
plt.grid(alpha=0.3)
plt.tight_layout()
plt.show()

print("Inercias calculadas:")
for k, inertia in zip(K_range, inertias):
    print(f"k={k}: {inertia:.2f}")


### 4.2 Método de Silhouette


In [ ]:
# Calcular silhouette score para diferentes valores de k
silhouette_scores = []

for k in K_range:
    kmeans = KMeans(n_clusters=k, random_state=42, n_init=10)
    cluster_labels = kmeans.fit_predict(X_scaled)
    silhouette_avg = silhouette_score(X_scaled, cluster_labels)
    silhouette_scores.append(silhouette_avg)

# Visualizar silhouette scores
plt.figure(figsize=(10, 6))
plt.plot(K_range, silhouette_scores, 'ro-', linewidth=2, markersize=8)
plt.xlabel('Número de Clusters (k)', fontsize=12)
plt.ylabel('Silhouette Score', fontsize=12)
plt.title('Método de Silhouette - Elección de k óptimo', fontsize=14, fontweight='bold')
plt.grid(alpha=0.3)
plt.tight_layout()
plt.show()

# Encontrar k óptimo
optimal_k = K_range[np.argmax(silhouette_scores)]
print(f"\nSilhouette Scores:")
for k, score in zip(K_range, silhouette_scores):
    print(f"k={k}: {score:.4f}")

print(f"\n✓ k óptimo según Silhouette: {optimal_k} (score: {max(silhouette_scores):.4f})")


In [ ]:
# Usar el k óptimo (ajustar según resultados)
n_clusters = optimal_k  # O el valor que consideres óptimo

# Entrenar modelo K-Means
kmeans = KMeans(n_clusters=n_clusters, random_state=42, n_init=10)
cluster_labels = kmeans.fit_predict(X_scaled)

print(f"Modelo K-Means entrenado con {n_clusters} clusters")
print(f"Distribución de clusters:")
unique, counts = np.unique(cluster_labels, return_counts=True)
for cluster, count in zip(unique, counts):
    print(f"  Cluster {cluster}: {count} clientes ({count/len(cluster_labels)*100:.2f}%)")


## 6. Métricas de Evaluación


In [ ]:
# Calcular métricas
silhouette_avg = silhouette_score(X_scaled, cluster_labels)
davies_bouldin = davies_bouldin_score(X_scaled, cluster_labels)
inertia = kmeans.inertia_

print("=" * 50)
print("MÉTRICAS DEL MODELO K-MEANS")
print("=" * 50)
print(f"Número de clusters: {n_clusters}")
print(f"Silhouette Score:   {silhouette_avg:.4f}")
print(f"Davies-Bouldin:     {davies_bouldin:.4f}")
print(f"Inercia:            {inertia:.2f}")
print("=" * 50)


## 7. Visualización de Clusters (PCA)


In [ ]:
# Reducir dimensionalidad con PCA para visualización
pca = PCA(n_components=2, random_state=42)
X_pca = pca.fit_transform(X_scaled)

# Visualizar clusters en 2D
plt.figure(figsize=(12, 8))
scatter = plt.scatter(X_pca[:, 0], X_pca[:, 1], c=cluster_labels, 
                     cmap='viridis', s=50, alpha=0.6, edgecolors='k', linewidth=0.5)

# Marcar centroides
centroids_pca = pca.transform(kmeans.cluster_centers_)
plt.scatter(centroids_pca[:, 0], centroids_pca[:, 1], 
           marker='X', s=200, c='red', label='Centroides', edgecolors='black', linewidth=2)

plt.xlabel(f'Primer Componente Principal ({pca.explained_variance_ratio_[0]*100:.1f}%)', fontsize=12)
plt.ylabel(f'Segundo Componente Principal ({pca.explained_variance_ratio_[1]*100:.1f}%)', fontsize=12)
plt.title('Visualización de Clusters - K-Means (PCA)', fontsize=14, fontweight='bold')
plt.colorbar(scatter, label='Cluster')
plt.legend()
plt.grid(alpha=0.3)
plt.tight_layout()
plt.show()


## 8. Interpretación de Clusters (Perfiles)


In [ ]:
# Agregar labels de cluster al dataframe original
df_clustered = df.copy()
df_clustered['Cluster'] = cluster_labels

# Calcular estadísticas por cluster
cluster_profiles = df_clustered.groupby('Cluster')[numeric_cols].mean()

print("PERFILES DE CLUSTERS (Promedios):")
print("=" * 80)
print(cluster_profiles.round(2))


In [ ]:
# Visualizar perfiles de clusters (algunas variables clave)
key_features = ['BALANCE', 'PURCHASES', 'CREDIT_LIMIT', 'PAYMENTS', 'TENURE']

fig, axes = plt.subplots(2, 3, figsize=(18, 10))
axes = axes.flatten()

for idx, feature in enumerate(key_features):
    if idx < len(axes):
        cluster_profiles[feature].plot(kind='bar', ax=axes[idx], color='#f5576c')
        axes[idx].set_title(f'{feature} por Cluster', fontsize=12, fontweight='bold')
        axes[idx].set_xlabel('Cluster', fontsize=10)
        axes[idx].set_ylabel('Valor Promedio', fontsize=10)
        axes[idx].tick_params(axis='x', rotation=0)
        axes[idx].grid(alpha=0.3, axis='y')

# Ocultar subplot extra
if len(key_features) < len(axes):
    axes[-1].axis('off')

plt.tight_layout()
plt.show()


## 9. Guardar Modelos y Preprocesador


In [ ]:
# Guardar el modelo K-Means
with open('../modelos/kmeans_model.pkl', 'wb') as f:
    pickle.dump(kmeans, f)

# Guardar el scaler
with open('../modelos/credit_scaler.pkl', 'wb') as f:
    pickle.dump(scaler, f)

print("✓ Modelo K-Means guardado en '../modelos/kmeans_model.pkl'")
print("✓ Scaler guardado en '../modelos/credit_scaler.pkl'")


## 10. Aplicaciones Reales del Clustering

### Posibles aplicaciones:

1. **Segmentación de Clientes**: Identificar grupos de clientes con comportamientos similares para estrategias de marketing personalizadas.

2. **Gestión de Riesgo**: Clasificar clientes según su perfil de uso de crédito para evaluar riesgos.

3. **Ofertas Personalizadas**: Crear ofertas específicas para cada segmento de clientes.

4. **Análisis de Comportamiento**: Entender patrones de compra y pago para mejorar servicios.

5. **Retención de Clientes**: Identificar clusters de alto valor para estrategias de retención.
